In [8]:
import openai
import os
from dotenv import load_dotenv

# --- Configuration ---
# Replace with your actual OpenAI API key
# It's best to set this as an environment variable for security
# For example: openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI()

# --- 1. Get Topic from User ---
topic = input("Enter a topic you want an explanation for: ")

# --- 2. Generate Text with OpenAI ---
explanation_text = None
try:
    print(f"\n🤖 Generating explanation for '{topic}'...")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Or your preferred model like "gpt-4o"
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides clear and simple explanations."},
            {"role": "user", "content": f"Explain '{topic}' in a simple way, in about 2-4 sentences."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    explanation_text = response.choices[0].message.content.strip()
    print(f"\n📜 Explanation:\n{explanation_text}")
except Exception as e:
    print(f"Error generating text with OpenAI: {e}")


🤖 Generating explanation for 'What is the meaning of life?'...

📜 Explanation:
The meaning of life is a big question that people have been trying to answer for a long time. It's all about finding your own purpose or reason for being here. For some, it might be about being happy, helping others, or making a difference in the world. Ultimately, the meaning of life is something each person has to figure out for themselves.


In [12]:
import playsound
from pathlib import Path

if explanation_text:
    print("\n🔊 Converting text to OpenAI speech...")
    speech_file_path = Path("~/projects/gruene/podcast/explanation_openai.mp3")

    try:
        # Available voices: alloy, echo, fable, onyx, nova, shimmer
        # Available models: tts-1, tts-1-hd (HD is higher quality, potentially higher latency/cost)
        # You can also choose `gpt-4o-mini-tts` as a model
        response = client.audio.speech.create(
            model="tts-1",         # or "tts-1-hd" or "gpt-4o-mini-tts"
            voice="alloy",         # Choose your preferred voice
            input=explanation_text,
            # response_format="mp3" # mp3 is default. Other options: opus, aac, flac, wav, pcm
        )

        # Stream the audio to a file
        response.stream_to_file(speech_file_path)
        print(f"Audio saved as {speech_file_path}")

        # --- 4. Play the Voice Line ---
        print("Playing audio...")
        playsound.playsound(str(speech_file_path)) # playsound needs a string path
        print("Playback finished.")

        # Optional: Clean up the audio file
        # os.remove(speech_file_path)
        # print(f"Temporary audio file {speech_file_path} removed.")

    except Exception as e:
        print(f"Error with OpenAI TTS or playing sound: {e}")
        if "401" in str(e): # Basic check for authentication error
             print("This might be an issue with your OpenAI API key or organization setup for TTS.")
        elif "Unsupported olha" in str(e) or "Unsupported model" in str(e): # Example for model/voice error
             print("Please ensure the selected voice and model for TTS are supported by your API access.")


else:
    print("No explanation text to convert to speech.")


🔊 Converting text to OpenAI speech...
Error with OpenAI TTS or playing sound: [Errno 2] No such file or directory: '~/projects/gruene/podcast/explanation_openai.mp3'


/tmp/ipykernel_220351/2683817295.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)
